In [1]:
import pandas as pd

In [42]:
matches= pd.read_csv("/Users/ayaanhaq/Documents/Machine Learning/BundesligaMatchPredictor/Bundesliga ML/Bundesliga Match Data/match_data_expanded.csv", index_col=0)

In [31]:
matches.shape

(612, 24)

In [32]:
matches["Team"].value_counts()

Team
Bayern Munich          34
Leverkusen             34
Holstein Kiel          34
Heidenheim             34
Hoffenheim             34
St. Pauli              34
Union Berlin           34
Augsburg               34
Wolfsburg              34
Monchengladbach        34
Stuttgart              34
Werder Bremen          34
RB Leipzig             34
Mainz 05               34
Freiburg               34
Dortmund               34
Eintracht Frankfurt    34
Bochum                 34
Name: count, dtype: int64

In [33]:
matches["Round"].value_counts()

Round
Matchweek 1     18
Matchweek 26    18
Matchweek 20    18
Matchweek 21    18
Matchweek 22    18
Matchweek 23    18
Matchweek 24    18
Matchweek 25    18
Matchweek 27    18
Matchweek 2     18
Matchweek 28    18
Matchweek 29    18
Matchweek 30    18
Matchweek 31    18
Matchweek 32    18
Matchweek 33    18
Matchweek 19    18
Matchweek 18    18
Matchweek 17    18
Matchweek 16    18
Matchweek 15    18
Matchweek 14    18
Matchweek 13    18
Matchweek 12    18
Matchweek 11    18
Matchweek 10    18
Matchweek 9     18
Matchweek 8     18
Matchweek 7     18
Matchweek 6     18
Matchweek 5     18
Matchweek 4     18
Matchweek 3     18
Matchweek 34    18
Name: count, dtype: int64

In [54]:
matches["Date"]=pd.to_datetime(matches["Date"])

In [36]:
matches.dtypes

Date          datetime64[ns]
Time                  object
Round                 object
Day                   object
Venue                 object
Result                object
GF                     int64
GA                     int64
Opponent              object
xG                   float64
xGA                  float64
Poss                   int64
Attendance            object
Captain               object
Formation             object
Referee               object
Sh                     int64
SoT                    int64
Dist                 float64
FK                     int64
PK                     int64
PKatt                  int64
Season                 int64
Team                  object
dtype: object

In [44]:
matches["Venue_code"]=matches["Venue"].astype("category").cat.codes

In [45]:
matches["Opp_code"]=matches["Opponent"].astype("category").cat.codes

In [58]:
matches

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,FK,PK,PKatt,Season,Team,Venue_code,Opp_code,hour,day_code,target
1,2024-08-25,15:30 (19:00),Matchweek 1,Sun,Away,W,3,2,Wolfsburg,2.4,...,0,0,0,2023,Bayern Munich,0,17,15,6,1
2,2024-09-01,17:30 (21:00),Matchweek 2,Sun,Home,W,2,0,Freiburg,1.5,...,0,1,1,2023,Bayern Munich,1,5,17,6,1
3,2024-09-14,18:30 (22:00),Matchweek 3,Sat,Away,W,6,1,Holstein Kiel,3.2,...,0,1,1,2023,Bayern Munich,0,9,18,5,1
4,2024-09-21,15:30 (19:00),Matchweek 4,Sat,Away,W,5,0,Werder Bremen,3.0,...,1,0,0,2023,Bayern Munich,0,16,15,5,1
5,2024-09-28,18:30 (22:00),Matchweek 5,Sat,Home,D,1,1,Leverkusen,1.4,...,1,0,0,2023,Bayern Munich,1,10,18,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,2025-04-19,15:30 (19:00),Matchweek 30,Sat,Away,L,0,1,Werder Bremen,0.7,...,2,0,0,2023,Bochum,0,16,15,5,0
609,2025-04-27,15:30 (19:00),Matchweek 31,Sun,Home,D,1,1,Union Berlin,1.5,...,0,0,1,2023,Bochum,1,15,15,6,0
610,2025-05-02,20:30 (00:00),Matchweek 32,Fri,Away,D,0,0,Heidenheim,1.3,...,0,0,0,2023,Bochum,0,7,20,4,0
611,2025-05-10,15:30 (19:00),Matchweek 33,Sat,Home,L,1,4,Mainz 05,0.9,...,0,0,0,2023,Bochum,1,11,15,5,0


In [47]:
matches["hour"]=matches["Time"].str.replace(":.+","",regex=True).astype("int")

In [55]:
matches["day_code"] = matches["Date"].dt.day_of_week

In [57]:
matches["target"]=(matches["Result"]=="W").astype("int")

In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
rf= RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [61]:
train= matches[matches["Date"]<"2025-01-01"]

In [62]:
test=matches[matches["Date"]>"2025-01-01"]

In [63]:
predictors=["Venue_code","Opp_code","hour", "day_code"]

In [64]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [65]:
preds=rf.predict(test[predictors])

In [66]:
from sklearn.metrics import accuracy_score

In [70]:
acc= accuracy_score(test["target"], preds)

In [71]:
acc

0.6198830409356725

In [72]:
combined= pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [73]:
pd.crosstab(index=combined['actual'], columns= combined ['prediction'])

prediction,0,1
actual,,
0,165,53
1,77,47


In [74]:
from sklearn.metrics import precision_score

In [75]:
precision_score(test["target"], preds)

0.47

In [77]:
grouped_matches= matches.groupby("Team")

In [78]:
group = grouped_matches.get_group("Bayern Munich")

In [79]:
group

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,FK,PK,PKatt,Season,Team,Venue_code,Opp_code,hour,day_code,target
1,2024-08-25,15:30 (19:00),Matchweek 1,Sun,Away,W,3,2,Wolfsburg,2.4,...,0,0,0,2023,Bayern Munich,0,17,15,6,1
2,2024-09-01,17:30 (21:00),Matchweek 2,Sun,Home,W,2,0,Freiburg,1.5,...,0,1,1,2023,Bayern Munich,1,5,17,6,1
3,2024-09-14,18:30 (22:00),Matchweek 3,Sat,Away,W,6,1,Holstein Kiel,3.2,...,0,1,1,2023,Bayern Munich,0,9,18,5,1
4,2024-09-21,15:30 (19:00),Matchweek 4,Sat,Away,W,5,0,Werder Bremen,3.0,...,1,0,0,2023,Bayern Munich,0,16,15,5,1
5,2024-09-28,18:30 (22:00),Matchweek 5,Sat,Home,D,1,1,Leverkusen,1.4,...,1,0,0,2023,Bayern Munich,1,10,18,5,0
6,2024-10-06,17:30 (21:00),Matchweek 6,Sun,Away,D,3,3,Eint Frankfurt,2.2,...,2,0,0,2023,Bayern Munich,0,4,17,6,0
7,2024-10-19,18:30 (22:00),Matchweek 7,Sat,Home,W,4,0,Stuttgart,3.2,...,0,0,0,2023,Bayern Munich,1,14,18,5,1
8,2024-10-27,15:30 (20:00),Matchweek 8,Sun,Away,W,5,0,Bochum,2.2,...,1,0,0,2023,Bayern Munich,0,2,15,6,1
9,2024-11-02,15:30 (20:00),Matchweek 9,Sat,Home,W,3,0,Union Berlin,2.5,...,0,1,1,2023,Bayern Munich,1,15,15,5,1
10,2024-11-09,15:30 (20:00),Matchweek 10,Sat,Away,W,1,0,St. Pauli,0.8,...,0,0,0,2023,Bayern Munich,0,13,15,5,1


In [80]:
def rolling_averages(group, cols, new_cols):
    group=group.sort_values("Date")
    rolling_stats= group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset= new_cols)
    return group

In [81]:
cols= ["GF","GA","Sh","SoT","Dist","FK","PK", "PKatt"]
new_cols= [f"{c}_rolling" for c in cols]

In [82]:
new_cols

['GF_rolling',
 'GA_rolling',
 'Sh_rolling',
 'SoT_rolling',
 'Dist_rolling',
 'FK_rolling',
 'PK_rolling',
 'PKatt_rolling']

In [83]:
rolling_averages(group, cols, new_cols)

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,day_code,target,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,FK_rolling,PK_rolling,PKatt_rolling
4,2024-09-21,15:30 (19:00),Matchweek 4,Sat,Away,W,5,0,Werder Bremen,3.0,...,5,1,3.666667,1.000000,16.000000,7.000000,14.333333,0.000000,0.666667,0.666667
5,2024-09-28,18:30 (22:00),Matchweek 5,Sat,Home,D,1,1,Leverkusen,1.4,...,5,0,4.333333,0.333333,19.333333,6.666667,16.400000,0.333333,0.666667,0.666667
6,2024-10-06,17:30 (21:00),Matchweek 6,Sun,Away,D,3,3,Eint Frankfurt,2.2,...,6,0,4.000000,0.666667,21.666667,7.000000,17.500000,0.666667,0.333333,0.333333
7,2024-10-19,18:30 (22:00),Matchweek 7,Sat,Home,W,4,0,Stuttgart,3.2,...,5,1,3.000000,1.333333,22.333333,7.666667,18.033333,1.333333,0.000000,0.000000
8,2024-10-27,15:30 (20:00),Matchweek 8,Sun,Away,W,5,0,Bochum,2.2,...,6,1,2.666667,1.333333,21.333333,8.666667,17.566667,1.000000,0.000000,0.000000
9,2024-11-02,15:30 (20:00),Matchweek 9,Sat,Home,W,3,0,Union Berlin,2.5,...,5,1,4.000000,1.000000,21.666667,11.000000,16.800000,1.000000,0.000000,0.000000
10,2024-11-09,15:30 (20:00),Matchweek 10,Sat,Away,W,1,0,St. Pauli,0.8,...,5,1,4.000000,0.000000,18.666667,8.666667,16.500000,0.333333,0.333333,0.333333
11,2024-11-22,20:30 (01:00),Matchweek 11,Fri,Home,W,3,0,Augsburg,3.7,...,4,1,3.000000,0.000000,15.666667,7.333333,17.533333,0.333333,0.333333,0.333333
12,2024-11-30,18:30 (23:00),Matchweek 12,Sat,Away,D,1,1,Dortmund,1.5,...,5,0,2.333333,0.000000,19.666667,8.000000,17.900000,0.333333,1.000000,1.000000
13,2024-12-07,15:30 (20:00),Matchweek 13,Sat,Home,W,4,2,Heidenheim,2.4,...,5,1,1.666667,0.333333,19.333333,7.666667,18.833333,1.000000,0.666667,0.666667


In [84]:
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))

/var/folders/zz/tz7yr7rs1cl2y022b2vb_0640000gn/T/ipykernel_39579/578198639.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [86]:
matches_rolling = matches_rolling.droplevel("Team")

In [88]:
matches_rolling.index = range(matches_rolling.shape[0])

In [89]:
matches_rolling

,Date,Time,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,day_code,target,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,FK_rolling,PK_rolling,PKatt_rolling
0,2024-09-20,20:30 (00:00),Matchweek 4,Fri,Home,L,2,3,Mainz 05,2.4,...,4,0,1.666667,2.333333,14.666667,3.666667,16.833333,0.666667,0.0,0.000000
1,2024-09-28,15:30 (19:00),Matchweek 5,Sat,Away,L,0,4,RB Leipzig,1.1,...,5,0,1.666667,2.666667,20.666667,5.000000,16.700000,0.666667,0.0,0.000000
2,2024-10-04,20:30 (00:00),Matchweek 6,Fri,Home,W,2,1,Gladbach,1.3,...,4,1,1.666667,2.666667,17.666667,5.000000,15.900000,0.000000,0.0,0.333333
3,2024-10-19,15:30 (19:00),Matchweek 7,Sat,Away,L,1,3,Freiburg,0.6,...,5,0,1.333333,2.666667,16.666667,4.000000,16.233333,0.000000,0.0,0.333333
4,2024-10-26,15:30 (19:00),Matchweek 8,Sat,Home,W,2,1,Dortmund,0.6,...,5,1,1.000000,2.666667,10.000000,2.666667,17.966667,0.000000,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,2025-04-19,15:30 (19:00),Matchweek 30,Sat,Away,D,2,2,Mainz 05,1.6,...,5,0,0.666667,1.666667,13.333333,3.666667,14.433333,0.333333,0.0,0.000000
551,2025-04-26,15:30 (19:00),Matchweek 31,Sat,Home,L,0,1,Freiburg,0.3,...,5,0,1.333333,2.000000,15.000000,4.333333,14.400000,0.333333,0.0,0.000000
552,2025-05-03,18:30 (22:00),Matchweek 32,Sat,Away,L,0,4,Dortmund,0.7,...,5,0,1.333333,2.000000,13.333333,3.666667,17.066667,0.666667,0.0,0.000000
553,2025-05-09,20:30 (00:00),Matchweek 33,Fri,Home,D,2,2,Hoffenheim,1.0,...,4,0,0.666667,2.333333,11.333333,2.333333,18.100000,0.666667,0.0,0.000000


In [90]:
def make_predictions(data, predictors):
    train = data[data["Date"]<"2025-01-01"]
    test= data[data["Date"]>"2025-01-01"]
    rf.fit(train[predictors], train["target"])
    preds= rf.predict(test[predictors])
    combined= pd.DataFrame(dict(actual=test["target"], predicted= preds), index=test.index)
    precision= precision_score(test["target"], preds)
    return combined, precision

In [91]:
combined, precision = make_predictions(matches_rolling, predictors+ new_cols)

In [92]:
precision

0.30303030303030304

In [93]:
combined = combined.merge(matches_rolling[["Date", "Team", "Opponent", "Result"]], left_index=True, right_index=True)

In [94]:
combined

,actual,predicted,Date,Team,Opponent,Result
12,0,0,2025-01-12,Augsburg,Stuttgart,L
13,1,0,2025-01-15,Augsburg,Union Berlin,W
14,1,0,2025-01-19,Augsburg,Werder Bremen,W
15,1,1,2025-01-25,Augsburg,Heidenheim,W
16,0,0,2025-02-01,Augsburg,St. Pauli,D
...,...,...,...,...,...,...
550,0,0,2025-04-19,Wolfsburg,Mainz 05,D
551,0,0,2025-04-26,Wolfsburg,Freiburg,L
552,0,0,2025-05-03,Wolfsburg,Dortmund,L
553,0,0,2025-05-09,Wolfsburg,Hoffenheim,D


In [97]:
merged= combined.merge(combined, left_on=["Date", "Team"], right_on=["Date", "Opponent"])

In [98]:
merged

,actual_x,predicted_x,Date,Team_x,Opponent_x,Result_x,actual_y,predicted_y,Team_y,Opponent_y,Result_y
0,0,0,2025-01-12,Augsburg,Stuttgart,L,1,0,Stuttgart,Augsburg,W
1,1,0,2025-01-15,Augsburg,Union Berlin,W,0,1,Union Berlin,Augsburg,L
2,1,0,2025-01-19,Augsburg,Werder Bremen,W,0,0,Werder Bremen,Augsburg,L
3,1,1,2025-01-25,Augsburg,Heidenheim,W,0,1,Heidenheim,Augsburg,L
4,0,0,2025-02-01,Augsburg,St. Pauli,D,0,1,St. Pauli,Augsburg,D
...,...,...,...,...,...,...,...,...,...,...,...
299,0,0,2025-04-19,Wolfsburg,Mainz 05,D,0,0,Mainz 05,Wolfsburg,D
300,0,0,2025-04-26,Wolfsburg,Freiburg,L,1,0,Freiburg,Wolfsburg,W
301,0,0,2025-05-03,Wolfsburg,Dortmund,L,1,0,Dortmund,Wolfsburg,W
302,0,0,2025-05-09,Wolfsburg,Hoffenheim,D,0,0,Hoffenheim,Wolfsburg,D


In [100]:
merged[(merged["predicted_x"]==1) & (merged["predicted_y"]==0)]["actual_x"].value_counts()

actual_x
0    34
1    18
Name: count, dtype: int64

In [101]:
13/27

0.48148148148148145